## Predict 2017 Sept Sea Ice Minimum

Use ridge and lasso regularization.  Validate the choice

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
import os
import sklearn.linear_model as lm


## Read in sea ice data and transform it to a wide form

In [5]:
sie = pd.read_csv(os.path.join(os.environ["HOME"], "tyyq", "infx574", "data", "N_extent_v2.1.csv.gz"))
sie = sie.rename(columns=lambda x: x.strip())
sie.set_value(sie["extent"] < 0, "extent", np.nan)
sie.set_value(sie["area"] < 0, "area", np.nan)
print("Original data description\n" + str(sie.describe()))

years = np.sort(sie["year"].unique())
print(str(len(years)) + "unique years:\n" + str(years))
data = pd.DataFrame(np.ones(len(years)), columns=["const"],
                    index = years)

for m in range(1,13):
    md = sie["extent"][sie["mo"] == m].values
    idx = sie["year"][sie["mo"] == m].values
    data["ext"+str(m)] = pd.Series(md, index = idx)
data["year"] = years
print(data.head(3))
print(data.tail(3))    


FileNotFoundError: [Errno 2] No such file or directory: '/home/kriarvi95/tyyq/infx574/data/N_extent_v2.1.csv.gz'

## First create training and test data.

Use years 28 years from 1979-2016 for training and the leftover for
testing.  We will do the final prediction on 2017.  We will have 4
closely correlated features: extent in Jan, Feb, March, April as
predictors.  We also have years, so we can introduce (potentially
                                                      non-linear)
trends, and we can create more features, such as $ext3^2$, or
$ext2^ext1$. 

In [ ]:
## shuffle the test/train years, pick first 28 for training, remainder
## for testing
years = np.arange(1979,2017)
np.random.shuffle(years)
yearTr = years[:28]
yearTe = years[28:]
print "training years", yearTr
print "test years", yearTe

In [ ]:
## include all 4 monthly extents as regressors
v = ["ext1", "ext2", "ext3", "ext4", "year"]
# include yearly trend
x_tr = data.loc[yearTr].as_matrix(v)
x_tr = np.hstack((x_tr, x_tr*x_tr))
# add quadratic terms, including for year.  Note: this may lead to overfitting!
y_tr = data.loc[yearTr].as_matrix(["ext9"])
# print "raw training data:\n"
# print x_tr
x_tst = data.loc[yearTe].as_matrix(v)
x_tst = np.hstack((x_tst, x_tst*x_tst))
y_tst = data.loc[yearTe].as_matrix(["ext9"])
# print "raw test data:\n"
# print x_tst
x_2017 = data.loc[2017].as_matrix(v)
x_2017 = np.hstack((x_2017, x_2017*x_2017))
# remove nan-s:
i = np.isnan(x_tr).any(axis=1)
j = np.isnan(y_tr).any(axis=1)
x_tr = x_tr[~(i | j)]
y_tr = y_tr[~(i | j)]
i = np.isnan(x_tst).any(axis=1)
j = np.isnan(y_tst).any(axis=1)
x_tst = x_tst[~(i | j)]
y_tst = y_tst[~(i | j)]

## Now let's normalize features
def normalize(x):
    mu = np.mean(x, axis = 0)
    mu = np.expand_dims(mu, axis=0)
    sd = np.std(x, axis = 0)
    sd = np.expand_dims(sd, axis =0)
    xn = (x - mu)/sd
    return xn, mu, sd

nx_tr, mu, sd = normalize(x_tr)
# print "feature means:\n"
# print mu
# print "standard deviations:\n"
# print sd
nx_tst = (x_tst - mu)/sd
# print "normalized training data:\n"
# print nx_tr
nx_2017 = (x_2017 - mu)/sd

## let's estimate the model (on normalized data)
# m = lm.LinearRegression()
# m = lm.Ridge(0.3)
m = lm.Lasso(0.1)
f = m.fit(nx_tr, y_tr)
print "\nEstimated bias:\n", f.intercept_
print "Estimated coefficients:\n", f.coef_
yHat = f.predict(nx_tst)
RMSE = np.sqrt((y_tst - yHat)**2).mean()
print "RMSE", RMSE
plt.scatter(y_tst, yHat)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("September Ice Extent Minimum (km^2)")
x = np.array([3,8])
plt.plot(x, x)
plt.show()

## And predict for 2017:
print "predicted for 2017:", f.predict(nx_2017)